In [1]:
%matplotlib inline
import pandas
import sklearn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from torch.utils.data import Dataset
from IPython.display import display
from collections import defaultdict
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

In [3]:
display(data_df.head())
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


In [4]:
data_df = data_df[["Sentence Utterance", "Object"]]
data_df.columns = ['input', 'raw_label']
display(data_df.describe())
display(data_df.raw_label.unique())

,input,raw_label
count,16175,16175
unique,13389,33
top,บริการอื่นๆ,service
freq,97,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

In [5]:
data_df['clean_label']=data_df['raw_label'].str.lower().copy()
data_df.drop('raw_label', axis=1, inplace=True)
display(data_df.describe())
display(data_df.clean_label.unique())

,input,clean_label
count,16175,16175
unique,13389,26
top,บริการอื่นๆ,service
freq,97,2528


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [6]:
data_df = data_df.drop_duplicates("input", keep="first")
data_df = data_df.reset_index(drop=True)
display(data_df.describe())

,input,clean_label
count,13389,13389
unique,13389,26
top,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,service
freq,1,2111


In [7]:
data_df['clean_label'].value_counts()

clean_label
service            2111
package            1797
internet           1790
balance            1482
promotion          1145
suspend             730
payment             641
phone_issues        581
bill                540
detail              327
information         297
mobile_setting      281
truemoney           248
nontruemove         246
roaming             246
lost_stolen         231
idd                 206
credit              173
ringtone             79
loyalty_card         67
balance_minutes      50
garbage              49
rate                 36
iservice             22
officer              10
contact               4
Name: count, dtype: int64

In [23]:
# remove class `contact` as it has only 4 sample (cannot do train 80 val 10 test 10)
data_df = data_df[data_df.clean_label != 'contact']

# Model 1 TF-IDF

Build a model to train a tf-idf text classifier. Use a simple logistic regression model for the classifier.

For this part, you may find this [tutorial](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html#sphx-glr-auto-examples-text-plot-document-classification-20newsgroups-py) helpful.

Below are some design choices you need to consider to accomplish this task. Be sure to answer them when you submit your model.

What tokenizer will you use? Why?

**Ans:**
`attacut`, because it is fast and accurate for Thai text. Although it is not the best tokenizer for Thai text, it is good enough for this task. The code below show comparison in time between `attacut` vs `deepcut` tokenizers. `deepcut` is the "best" tokenizer for Thai text, but it is a lot slower than `attacut`.

Will you ignore some stop words (a, an, the, to, etc. for English) in your tf-idf? Is it important?
PythaiNLP provides a list of stopwords if you want to use (https://pythainlp.org/docs/2.0/api/corpus.html#pythainlp.corpus.common.thai_stopwords)

**Ans:**
From my experiment:
- Accuracy on test set including stop words: 65.273%, time used for inference: 0.472s
- Accuracy on test set excluding stop words: 60.119%, time used for inference: 0.730s

> Note that the stop words used are sole from `pythainlp.corpus.common.thai_stopwords`. `PHONE_NUMBER_REMOVED` was NOT added to the stop words list as it occurred only 401 times from 13389 samples (it must have some correlation to label class).

The dictionary of TF-IDF is usually based on the training data. How many words in the test set are OOVs?

**Ans:**
- 751 words in the test set are OOVs (not found in the training set's vocabulary).

In [9]:
import deepcut
import attacut
import time
from tqdm import tqdm
import pythainlp

In [117]:
# 19 secs for 100 samples
# 13389 samples = 42 mins; ain't nobody got time for that
start_time = time.time()
for i in range(100):
    # print(f"original: {data_df['input'][i]}")
    # print(deepcut.tokenize(data_df['input'][i]))
    deepcut.tokenize(data_df['input'][i])
end_time = time.time()

print(f"Total time: {end_time - start_time}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step 
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
3/3 ━━━━━━━

In [118]:
# 2.25 secs for 100 samples
# 13389 samples = 5 mins; now we're talking
start_time = time.time()
for i in range(100):
    attacut.tokenize(data_df['input'][0])
end_time = time.time()

print(f"Total time: {end_time - start_time}")


/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/attacut/models/__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_d

Total time: 3.103543996810913


In [10]:
data = data_df.copy().to_numpy()

unique_label = data_df.clean_label.unique()

label_2_num_map = dict(zip(unique_label, range(len(unique_label))))
num_2_label_map = dict(zip(range(len(unique_label)), unique_label))

print("Create Mappings")
display(num_2_label_map)
display(label_2_num_map)

# print("Before Mappings")
# display(data[:, 1])
data[:,1] = np.vectorize(label_2_num_map.get)(data[:,1])
# print("After Mappings")
# display(data[:, 1])

Create Mappings


{0: 'payment',
 1: 'package',
 2: 'suspend',
 3: 'internet',
 4: 'phone_issues',
 5: 'service',
 6: 'nontruemove',
 7: 'balance',
 8: 'detail',
 9: 'bill',
 10: 'credit',
 11: 'promotion',
 12: 'mobile_setting',
 13: 'iservice',
 14: 'roaming',
 15: 'truemoney',
 16: 'information',
 17: 'lost_stolen',
 18: 'balance_minutes',
 19: 'idd',
 20: 'garbage',
 21: 'ringtone',
 22: 'rate',
 23: 'loyalty_card',
 24: 'officer'}

{'payment': 0,
 'package': 1,
 'suspend': 2,
 'internet': 3,
 'phone_issues': 4,
 'service': 5,
 'nontruemove': 6,
 'balance': 7,
 'detail': 8,
 'bill': 9,
 'credit': 10,
 'promotion': 11,
 'mobile_setting': 12,
 'iservice': 13,
 'roaming': 14,
 'truemoney': 15,
 'information': 16,
 'lost_stolen': 17,
 'balance_minutes': 18,
 'idd': 19,
 'garbage': 20,
 'ringtone': 21,
 'rate': 22,
 'loyalty_card': 23,
 'officer': 24}

In [11]:
def strip_str(string):
    return string.strip()
     
# print("Before")
# print(data)
data[:,0] = np.vectorize(strip_str)(data[:,0])
# print("After")
# print(data)

In [12]:
data

array([['<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter Services เค้าเช็ต 3276.25 บาท เมื่อวานที่ผมเช็คที่ศูนย์บอกมียอด 3057.79 บาท',
        0],
       ['internet ยังความเร็วอยุ่เท่าไหร ครับ', 1],
       ['ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ', 2],
       ...,
       ['ยอดเงินเหลือเท่าไหร่ค่ะ', 7],
       ['ยอดเงินในระบบ', 7],
       ['สอบถามโปรโมชั่นปัจจุบันที่ใช้อยู่ค่ะ', 1]], dtype=object)

In [35]:
x = data[:, 0]
y = data[:, 1]
print(x.shape, y.shape)

(13385,) (13385,)


In [36]:
X_train: np.ndarray
y_train: np.ndarray
X_val: np.ndarray
y_val: np.ndarray
X_test: np.ndarray
y_test: np.ndarray

X_train, X_temp, y_train, y_temp = train_test_split(x, y, stratify=y, test_size=0.2,random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, stratify=y_temp, test_size=0.5, random_state=42)
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(10708,) (10708,) (1338,) (1338,) (1339,) (1339,)


In [38]:
vocab = defaultdict(lambda: len(vocab))
vocab["<UNK>"] = 0

token_count = defaultdict(int)

for i in tqdm(range(len(X_train))):
    tokens = attacut.tokenize(X_train[i])
    for token in tokens:
        vocab[token]
        token_count[token] += 1
    
# 4728 unique tokens in training set (+ 1 from UNK)
print(len(vocab))

100%|██████████| 10708/10708 [04:26<00:00, 40.23it/s]

4728


In [72]:
vocab_ = defaultdict(lambda: len(vocab_))
for i in tqdm(range(len(data))):
    tokens = attacut.tokenize(data[i][0])
    for token in tokens:
        vocab_[token]
# 5447 unique tokens in entire dataset
print(len(vocab_))

100%|██████████| 13389/13389 [04:25<00:00, 50.42it/s]

5447


In [18]:
oov_count = 0
for i in tqdm(range(len(X_test))):
    tokens = attacut.tokenize(X_test[i])
    for token in tokens:
        token_count[token] += 1
        if token not in vocab:
            oov_count += 1

# 373 unique tokens in test set but not in training set (OOV)
print(oov_count)

  0%|          | 0/1339 [00:00<?, ?it/s]/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/attacut/models/__init__.py:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this ex

373


In [19]:
# sort by frequency
sorted_token_count = sorted(token_count.items(), key=lambda x: x[1], reverse=True)
sorted_token_count[:100]

[(' ', 27095),
 ('ค่ะ', 6699),
 ('จะ', 6019),
 ('ครับ', 5646),
 ('ได้', 4253),
 ('ไม่', 4023),
 ('ใช้', 3735),
 ('สอบถาม', 3532),
 ('ผม', 3045),
 ('ว่า', 3034),
 ('แล้ว', 2744),
 ('ที่', 2567),
 ('ไป', 2476),
 ('นี้', 2440),
 ('อยาก', 2307),
 ('มา', 2079),
 ('มัน', 2053),
 ('เบอร์', 1974),
 ('ค่า', 1927),
 ('มี', 1901),
 ('บริการ', 1880),
 ('คะ', 1834),
 ('โทร', 1746),
 ('เงิน', 1685),
 ('ยัง', 1642),
 ('ของ', 1621),
 ('ให้', 1551),
 ('สมัคร', 1532),
 ('พอดี', 1508),
 ('เปิด', 1399),
 ('ขอ', 1302),
 ('พี่', 1292),
 ('ยอด', 1283),
 ('เป็น', 1251),
 ('โปรโมชั่น', 1177),
 ('งาน', 1172),
 ('การ', 1171),
 ('ยกเลิก', 1171),
 ('ชำระ', 1150),
 ('หน่อย', 1145),
 ('ต้องการ', 1116),
 ('อะไร', 1074),
 ('สัญญาณ', 1058),
 ('โทรศัพท์', 1058),
 ('คือ', 1042),
 ('ทราบ', 1040),
 ('แต่', 1027),
 ('อินเตอร์เน็ต', 1003),
 ('อ่ะ', 987),
 ('เลย', 986),
 ('เติม', 943),
 ('เรื่อง', 930),
 ('เปลี่ยน', 929),
 ('เข้า', 927),
 ('อยู่', 883),
 ('ต้อง', 881),
 ('เดือน', 852),
 ('กับ', 842),
 ('ตอน', 811),
 ('วัน', 8

In [39]:
from pythainlp.corpus.common import thai_stopwords

stopwords = thai_stopwords()

In [40]:
def string_to_token_ids(X: np.ndarray):
    new_X = np.zeros(X.shape, dtype=object)
    for i in tqdm(range(len(X))):
        token_ids = []
        tokens = attacut.tokenize(X[i])
        # should we remove any tokens?
        # this line later added to test if removing stopwords is necessary
        tokens = [token for token in tokens if token not in stopwords]

        for token in tokens:
            if token in vocab:
                token_ids.append(vocab[token])
            else:
                token_ids.append(vocab["<UNK>"])
        
        new_X[i] = token_ids
    
    return new_X

In [41]:
print(X_train[0])
X_train = string_to_token_ids(X_train)
X_test = string_to_token_ids(X_test)
print(X_train[0])

เดือนละ 150 บาทเล่นได้ทั้งวันหรือเปล่า


100%|██████████| 1339/1339 [00:34<00:00, 38.85it/s]

[1, 3, 4, 3, 5, 6, 11]


In [42]:
num_docs_with_w = defaultdict(int)
for i in tqdm(range(len(X_train))):
    unique_tokens = set(X_train[i])
    
    for token in unique_tokens:
        num_docs_with_w[token] += 1
# sort token idx
num_docs_with_w = dict(sorted(num_docs_with_w.items(), key=lambda x: x[0]))

idf = np.log(len(X_train) / np.array(list(num_docs_with_w.values())))

100%|██████████| 10708/10708 [00:00<00:00, 148213.71it/s]


In [43]:
len(idf)

4456

In [44]:
num_docs_with_w

{1: 597,
 3: 6921,
 4: 25,
 5: 344,
 6: 369,
 11: 271,
 15: 794,
 16: 1307,
 17: 1278,
 20: 156,
 21: 717,
 24: 404,
 25: 443,
 30: 19,
 31: 168,
 32: 269,
 38: 57,
 39: 1575,
 40: 1115,
 41: 534,
 42: 656,
 43: 724,
 44: 62,
 46: 15,
 48: 389,
 50: 549,
 53: 1,
 54: 4,
 55: 795,
 57: 838,
 59: 100,
 61: 151,
 62: 1010,
 66: 564,
 67: 1060,
 77: 9,
 79: 206,
 81: 395,
 83: 327,
 84: 426,
 85: 13,
 87: 52,
 88: 1,
 89: 172,
 90: 146,
 92: 368,
 93: 6,
 95: 3,
 96: 68,
 98: 265,
 101: 151,
 102: 13,
 103: 17,
 104: 277,
 106: 531,
 108: 10,
 111: 830,
 112: 438,
 115: 9,
 117: 2,
 118: 1,
 120: 732,
 121: 18,
 122: 3,
 124: 724,
 126: 233,
 127: 303,
 128: 41,
 129: 125,
 130: 20,
 131: 1,
 132: 377,
 134: 244,
 136: 79,
 137: 2,
 138: 14,
 141: 88,
 142: 166,
 143: 5,
 144: 1,
 145: 6,
 148: 18,
 149: 1,
 150: 183,
 151: 320,
 152: 6,
 154: 668,
 155: 138,
 157: 2524,
 159: 15,
 161: 58,
 162: 4,
 163: 2,
 166: 205,
 167: 1,
 168: 40,
 169: 338,
 171: 91,
 172: 390,
 174: 4,
 175: 1,
 1

In [45]:
idf

array([2.88682929, 0.43643086, 6.05987058, ..., 9.2787464 , 9.2787464 ,
       9.2787464 ])

In [46]:
def gen_tf_idf_for_sample(sample: list[int]):
    N = len(sample)
    tf = defaultdict(int)
    for token in sample:
        tf[token] += 1
    for token in tf:
        tf[token] = tf[token] / N
    
    tf_idf = np.zeros(len(vocab)) # vocab from only training set
    for token in tf:
        if token < len(idf)-1:
            tf_idf[token] = tf[token] * idf[token]
    
    return tf_idf

def gen_tf_idf(X: np.ndarray):
    new_X = np.zeros((X.shape[0], len(vocab)))
    for i, sample in enumerate(X):
        new_X[i] = gen_tf_idf_for_sample(sample)
    
    return new_X

In [47]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape, len(vocab))

(10708,) (1339,) (10708,) (1339,) 4728


In [48]:
X_train = gen_tf_idf(X_train)
X_test = gen_tf_idf(X_test)
y_train = y_train.copy().astype(int)
y_test = y_test.copy().astype(int)

In [49]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(10708, 4728) (1339, 4728) (10708,) (1339,)


In [50]:
# 100 max epochs took 2m 1s
# 100 max epochs with ignoring stop words took 2m 20s
logmodel = LogisticRegression(class_weight='balanced', max_iter=100, random_state=2025)
logmodel.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=2025)

In [ ]:
# 0.472s
start_time = time.time()
predictions = logmodel.predict(X_test)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

Total time: 0.47194814682006836


In [52]:
# ignoring stop words 0.730s
start_time = time.time()
predictions = logmodel.predict(X_test)
end_time = time.time()

print(f"Total time: {end_time - start_time}")

Total time: 0.7300021648406982


In [34]:
# 100 max epochs: acc 65.273% 
print("Model Acc. on test data %f%%"
       % ((y_test == predictions).sum() / y_test.shape[0] * 100))

Model Acc. on test data 65.272591%


In [53]:
# 100 max epochs with ignoring stop words: acc 60.119%
print("Model Acc. on test data %f%%"
       % ((y_test == predictions).sum() / y_test.shape[0] * 100))

Model Acc. on test data 60.119492%


In [244]:
report = classification_report(y_test, predictions, output_dict=True, digits=2)
report

/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/idhibhatpankam/Code/courses/NLP-SYS/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_

{'0': {'precision': 0.572289156626506,
  'recall': 0.7421875,
  'f1-score': 0.6462585034013606,
  'support': 128.0},
 '1': {'precision': 0.7096774193548387,
  'recall': 0.55,
  'f1-score': 0.6197183098591549,
  'support': 360.0},
 '2': {'precision': 0.7295597484276729,
  'recall': 0.7945205479452054,
  'f1-score': 0.760655737704918,
  'support': 146.0},
 '3': {'precision': 0.6873065015479877,
  'recall': 0.6201117318435754,
  'f1-score': 0.6519823788546255,
  'support': 358.0},
 '4': {'precision': 0.5029239766081871,
  'recall': 0.7413793103448276,
  'f1-score': 0.5993031358885017,
  'support': 116.0},
 '5': {'precision': 0.8379310344827586,
  'recall': 0.5758293838862559,
  'f1-score': 0.6825842696629213,
  'support': 422.0},
 '6': {'precision': 0.359375,
  'recall': 0.46938775510204084,
  'f1-score': 0.40707964601769914,
  'support': 49.0},
 '7': {'precision': 0.8857142857142857,
  'recall': 0.7306397306397306,
  'f1-score': 0.8007380073800738,
  'support': 297.0},
 '8': {'precision'